<h1> Algortimo de Identificación de Hotspots </h1>
<h4>Alberto Ángel Ramírez Aguilera </h4>
Instrucciones: Para ejecutar este cuaderno, es posible: <br>
1) EJECUTAR CADA CELDA POR SEPARADO CON EL BOTÓN RUN <br>
2) SELECCIONAR LA OPCIÓN CELL => RUN ALL

In [1]:
numClusteres = int(input("Ingrese el numero deseado de clusteres: "))

Ingrese el numero deseado de clusteres: 200


In [2]:
import pandas as pd
import numpy as np
import requests,json

Para el desarrollo de esta metodología, trabajaremos con los datos de las carpetas de investigación llevadas a cabo en la CDMX en el año 2020.

In [3]:
url = "https://datos.cdmx.gob.mx/api/records/1.0/search/?dataset=carpetas-de-investigacion-pgj-de-la-ciudad-de-mexico&rows=10000&facet=ao_hechos&facet=mes_hechos&facet=delito&facet=categoria_delito&facet=fiscalia&facet=alcaldia_hechos&facet=colonia_hechos&facet=ao_inicio&facet=mes_inicio&refine.ao_hechos=2020"

In [4]:
re = requests.get(url)

In [5]:
re = json.loads(re.text)["records"]

In [6]:
re = list(map(lambda x: x["fields"],re))

Convertimos ahora los datos obtenidos de la API de Datos Abiertos a un dataframe

In [7]:
df = pd.DataFrame.from_records(re)
df = df.dropna()
df["latitud"] = df["latitud"].astype(float)
df["longitud"] = df["longitud"].astype(float)

In [8]:
df.head()

,latitud,mes_hechos,mes_inicio,categoria_delito,fecha_inicio,calle_hechos,calle_hechos2,alcaldia_hechos,geopoint,fiscalia,agencia,unidad_investigacion,fecha_hechos,colonia_hechos,ao_hechos,delito,ao_inicio,longitud
0,19.50587,Enero,Enero,DELITO DE BAJO IMPACTO,01/01/2020 01:28,AV CENTENARIO,CANAL DE DESFOGUE Y AV. CENTENARIO,GUSTAVO A MADERO,"[19.50587, -99.09253]",INVESTIGACIÓN EN GUSTAVO A. MADERO,GAM-4,UI-2CD,01/01/2020 01:15,JUAN GONZÁLEZ ROMERO,2020,HOMICIDIO CULPOSO POR TRÁNSITO VEHICULAR (CAIDA),2020,-99.09253
1,19.49629,Enero,Enero,DELITO DE BAJO IMPACTO,01/01/2020 03:17,FRAY TORIBIO DE BENAVENTE,NA,GUSTAVO A MADERO,"[19.49629, -99.0913]",INVESTIGACIÓN EN GUSTAVO A. MADERO,GAM-4,UI-2CD,01/01/2020 02:29,VASCO DE QUIROGA,2020,AMENAZAS,2020,-99.09130
2,19.33196,Enero,Enero,DELITO DE BAJO IMPACTO,01/01/2020 04:35,CERRADA FRANCISCO VILLA,NA,ALVARO OBREGON,"[19.33196, -99.26737]","INVESTIGACIÓN PARA LA ATENCIÓN DE NIÑOS, NIÑAS...",57,UI-2CD,01/01/2020 02:15,SAN BARTOLO AMEYALCO,2020,ROBO A CASA HABITACION SIN VIOLENCIA,2020,-99.26737
3,19.52175,Enero,Enero,HOMICIDIO DOLOSO,01/01/2020 05:59,15 Y AVENIDA GUADALUPE,NA,GUSTAVO A MADERO,"[19.52175, -99.15694]",INVESTIGACIÓN PARA LA ATENCIÓN DEL DELITO DE H...,HOM-2,1 CON DETENIDO 1 C/D,01/01/2020 04:50,GUADALUPE PROLETARIA,2020,HOMICIDIO POR ARMA DE FUEGO,2020,-99.15694
4,19.37380,Enero,Enero,DELITO DE BAJO IMPACTO,01/01/2020 07:34,CALLE 22,NA,ALVARO OBREGON,"[19.3738, -99.20975]",INVESTIGACIÓN EN ÁLVARO OBREGÓN,AO-3,UI-2CD,01/01/2020 07:00,OLIVAR DEL CONDE 1A SECCIÓN,2020,LESIONES CULPOSAS POR TRANSITO VEHICULAR EN CO...,2020,-99.20975


Es necesario obtener las coordenadas de cada carpeta de investigación, por lo que las almacenamos en un arreglo

In [9]:
geopoints = np.array(list(df.geopoint))

Una vez teniendo todos las coordenadas de las carpetas de investigación, podemos agruparlas. <br>
Para esto utilizaremos un algoritmo especializado llamado k means.

In [10]:
from sklearn.cluster import KMeans


K Means recibe como parámetro el número de clusters que se desean identificar<br> Obtenemos entonces los clusteres asignados, así como los centorides de cada cluster

In [11]:
kmeans = KMeans(n_clusters=numClusteres, random_state=0).fit(geopoints)
labels = kmeans.labels_
centroides = list(map(lambda x: list(x),kmeans.cluster_centers_))


Integramos los clusteres a los que pertenece cada coordenada al dataset


In [12]:
df["cluster"] = labels
delitosCluster = np.array(list(df.groupby(['cluster']).size()))
colores = ["red" if x>np.average(delitosCluster)*1.5 else "blue" for x in delitosCluster]

Asimismo, obtendremos el número de delitos que se tiene por cada cluster, esto lo logramos agrupando los datos (tal y como lo hariamos en una consulta SQL, es decir, con la funcion groupby)
<br>
También, asginaremos a cada número de cluster un color: <br>
-<b>Rojo</b>, si el número de delitos en el cluster es mayor que el promedio de delitos <br>
-<b>Azul</b>, si el número de delitos en el cluster es menor que el promedio de delitos<br>
Con estos datos, podemos comenzar a trabajar con el mapa que se encargará de desplegar estos datos

In [13]:
import folium

Primero, creamos un mapa que estará centrado con base en las coordenadas recibidas <br>
Después, por cada cluster identificado, crearemos un circulo cuyo radio será el numoer de delitos en ese cluster dividio entre 10, y el color será rojo o azul, según lo explicado previamente <br>
<b>Para conocer el numero de delitos en el cluster, debemos de hacer click sobre el contorno del circulo representivo de ese cluster

In [14]:
map_ = folium.Map(location=[df["latitud"].mean(),df["longitud"].mean()], zoom_start = 11)
for point,color,delitos in zip(centroides,colores,delitosCluster):
    folium.CircleMarker(point,
                    radius=delitos/10,
                    color=color,
                   ).add_child(folium.Popup("Numero de delitos en cluster: "+str(delitos))).add_to(map_)

map_

Como conclusión, se puede observar que de manera general, la mayoria de los hot spots serán ubicados en la zona centro de la CDMX. Esto sin importar demasiado el número de cluster con los que se desee trabajar.